In [1]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd  

In [2]:
engine = create_engine(connstr) 
def get_df(query):
    conn = engine.connect() 
    df = pd.read_sql(query,con=conn) 
    conn.close()
    return df

In [3]:
def id_split(s):
    s = s + '000'
    s = re.sub('[^\d]', '', s)  
    return s[:4] 
#print( id_split('5.1-4') )

In [4]:
def id_num(s):
    if s == '':
        return 0
    s = s.strip().lower().replace(' ', '')
    l = ['a','b','c','d','e','f','g','h']
    d = [.100, 1, 100, 10000]
    v = [0,0,0,0]
    for i, c in enumerate(l):
        s = s.replace(c,f'{i}') 
    l = s.split('.')
    for i, c in enumerate(l):
        v[i] =  int(c) / d[i] 
    return sum(v)  
print(id_num('1.1.1')) 
print(id_num('1.1.10.1')) 

11.01
11.1001


In [13]:
def normalizedf(df):
    df.drop( columns=[c for c in df.columns if '__' in c ] , inplace=True )   
    cols = list(df)   
    df[cols] = df[cols].astype(str)
    df['IDNUM'] = df['IDTEXT'].apply(id_num)
    df.sort_values(['QGROUP','IDNUM'],inplace=True)   
    df['QPK'] = range(28500, len(df)+28500 )  
    df['sortpos'] = range(1, len(df)+1)  
    return df

In [14]:
query = f""" \
SELECT PK_Question, IDTEXT, QGROUP FROM vwQuestions 
WHERE PK_Form = '2021-A-HVA' AND SectionNum > 1 
ORDER BY SortPos Asc
"""  
df1  = get_df(query) 
df1 = normalizedf(df1)
df1.sort_values(['QGROUP','IDNUM'], inplace=True)
df1.to_csv(r'C:\Users\Tim\Documents\1802\HVA\2021.csv', index=False)  
df1[:] 

,PK_Question,IDTEXT,QGROUP,IDNUM,QPK,sortpos
0,22693,,2433,0.00,28500,1
23,22699,,2433,0.00,28501,2
29,22707,,2433,0.00,28502,3
5,22850,12,2433,120.00,28503,4
6,22851,12.1,2433,121.00,28504,5
11,22852,12.2,2433,122.00,28505,6
12,22853,12.3,2433,123.00,28506,7
16,22854,12.4,2433,124.00,28507,8
17,22855,12.5,2433,125.00,28508,9
18,22856,12.6,2433,126.00,28509,10


In [15]:
query = f""" \
SELECT PK_Question, IDTEXT, QGROUP FROM vwQuestions 
WHERE PK_Form = '2022-A-HVA' AND SectionNum > 1 
ORDER BY SortPos Asc
"""  
df2  = get_df(query) 
df2 = normalizedf(df2)
df2.sort_values(['QGROUP','IDNUM'], inplace=True)
df2.to_csv(r'C:\Users\Tim\Documents\1802\HVA\2022.csv', index=False)  
df2[:2] 

,PK_Question,IDTEXT,QGROUP,IDNUM,QPK,sortpos


In [23]:
df = pd.merge(df1, df2, how='outer', on=['IDTEXT'], suffixes=(None, '__')) 
df = normalizedf(df)
df.to_csv(r'C:\Users\Tim\Documents\1802\HVA\df_years_12.csv', index=False)  
df[0:2]  

,IDTEXT,QuestionText,QGROUP,IDNUM,QPK,sortpos
0,0.1,HVA Information,nan,1.0,28200,1
1,1,What is the purpose of the HVA?,2431.0,10.0,28201,2


In [60]:
query = f"""  SELECT TOP 1 
    PK_Question, FormName, FK_QuestionGroup,sortpos,identifier_text, FK_QuestionType
    , FK_InputType, FK_PickListType, WarningQuestion, ExternalLinkType, QuestionText, help_text 
    FROM fsma_Questions 
    """  
df_qmeta = get_df(query) 
cols = ','.join(df_qmeta.columns)
INS = f'INSERT INTO fsma_Questions ({cols}) VALUES '
print(INS)

INSERT INTO fsma_Questions (PK_Question,FormName,FK_QuestionGroup,sortpos,identifier_text,FK_QuestionType,FK_InputType,FK_PickListType,WarningQuestion,ExternalLinkType,QuestionText,help_text) VALUES 


In [120]:
sql = "" 
for i, r in dff[:].iterrows(): 
    s = "(@QID, @FormName, @PK_QGroup, @SORD, N'@IDTEXT', @QTYPE, 1, @PKTYPE, 0, NULL, N'@QTEXT','@HTEXT'),"
    s = s.replace('@QID',  str(r['QPK']))
    s = s.replace('@SORD', str(i+1)) 
    s = s.replace('@QTYPE', r['FK_QuestionType'])
    s = s.replace('@PKTYPE', r['FK_PickListType'])
    s = s.replace('@IDTEXT', r['IDTEXT']) 
    s = s.replace('@QTEXT', r['QuestionText'].replace('\'', '`').replace('\ufffd', ''))  
    s = s.replace('@HTEXT', r['help_text'])  
    sql = sql + s + '\n'
sql = sql.replace("'~')", "NULL)")    
# sql = sql.replace("N'NULL'", "N''")   
sql = sql.replace("~,", "NULL,")  
# sql = f'{INS}\n {sql}\n' 
print( sql )

(22693, @FormName, @PK_QGroup, 1, N'', 63, 1, NULL, 0, NULL, N'Modernization',NULL),
(22850, @FormName, @PK_QGroup, 2, N'12', 63, 1, NULL, 0, NULL, N'12.0 Does this system directly employ or operate in an environment that has adopted any of the following cybersecurity mitigation technologies?',NULL),
(22851, @FormName, @PK_QGroup, 3, N'12.1', 17, 1, NULL, 0, NULL, N'Adoption of application, data, and enterprise level protections such as Encrypting Data at Rest and Encrypting Data in Transit ',NULL),
(22852, @FormName, @PK_QGroup, 4, N'12.2', 17, 1, NULL, 0, NULL, N'Multi-Factor Authentication',NULL),
(22853, @FormName, @PK_QGroup, 5, N'12.3', 17, 1, NULL, 0, NULL, N'Least Privilege',NULL),
(22854, @FormName, @PK_QGroup, 6, N'12.4', 17, 1, NULL, 0, NULL, N'Application Whitelisting',NULL),
(22855, @FormName, @PK_QGroup, 7, N'12.5', 17, 1, NULL, 0, NULL, N'Mobile Device Management',NULL),
(22856, @FormName, @PK_QGroup, 8, N'12.6', 17, 1, NULL, 0, NULL, N'Cloud-oriented protection (If in a

In [ ]:
with open(fr'C:\Users\Tim\Documents\1802\HVA\update.sql', 'w') as f:
    f.write(sql)  



df = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='PurposePickList').fillna('NULL') 
cols = list(df) # Creates list of all column headers
df[cols] = df[cols].astype(str)
df['DisplayCode'] =  df['DisplayValue'].apply(lambda s: re.sub('[^A-Z0-9]','',s.upper()))
df['DisplayCode'] =  df['DisplayCode'].apply(lambda s: re.sub('[AEIOURST]','',s)[:8])
s =''
for i,r, in df.iterrows():
    PK_PickList=r['PK_PickList']
    DisplayCode=r['DisplayCode']
    DisplayValue=r['DisplayValue'] 
    if DisplayValue=='Other':
        DisplayCode='OTH'
    s = s + f"({PK_PickList},@PKT, N'{DisplayCode}',N'{DisplayValue}',{i}, GETDATE(), 1),\n"

print(s)
